In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
import re

# Load Data

In [127]:
def load_data(data_path="../data/"):
    """Load the credit scoring dataset."""
    # Check if the data directory exists
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"Data directory '{data_path}' not found.")
    
    # Look for CSV files in the data directory
    csv_files = [f for f in os.listdir(data_path) if f.endswith('train.csv')]
    
    if not csv_files:
        raise FileNotFoundError(f"No CSV files found in '{data_path}'.")
    
    # Load the first CSV file found
    file_path = os.path.join(data_path, csv_files[0])
    print(f"Loading data from: {file_path}")
    
    return pd.read_csv(file_path)


In [128]:
df = load_data()
df.head()

Loading data from: ../data/train.csv


/tmp/ipykernel_7900/948120084.py:17: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


# Remove Duplicates

In [129]:
duplicates = df[df.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [ID, Customer_ID, Month, Name, Age, SSN, Occupation, Annual_Income, Monthly_Inhand_Salary, Num_Bank_Accounts, Num_Credit_Card, Interest_Rate, Num_of_Loan, Type_of_Loan, Delay_from_due_date, Num_of_Delayed_Payment, Changed_Credit_Limit, Num_Credit_Inquiries, Credit_Mix, Outstanding_Debt, Credit_Utilization_Ratio, Credit_History_Age, Payment_of_Min_Amount, Total_EMI_per_month, Amount_invested_monthly, Payment_Behaviour, Monthly_Balance, Credit_Score]
Index: []

[0 rows x 28 columns]


# Drop Columns

In [130]:
def drop_columns(df, columns_to_drop):
    # Drop columns, ignoring errors if a column does not exist
    df = df.drop(columns=columns_to_drop, axis=1, errors='ignore')

    # Verify and print confirmation for dropped columns
    for col in columns_to_drop:
        if col not in df.columns:
            print(f"Column '{col}' dropped successfully.")

    return df  

In [131]:
columns_to_drop = [
    "ID",
    "Name",  
    "SSN",  
    "Customer_ID",  
]

df = drop_columns(df, columns_to_drop)  

Column 'ID' dropped successfully.
Column 'Name' dropped successfully.
Column 'SSN' dropped successfully.
Column 'Customer_ID' dropped successfully.


# Correct Data Types

In [132]:
df.dtypes

Month                        object
Age                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
Credit_Score                 object
dtype: object

In [133]:
# Annual_Income (object → float64)
df['Annual_Income'] = df['Annual_Income'].astype(str).str.replace(r'[^\d.]', '', regex=True).astype(float)

# Num_of_Loan (object → int64)
df['Num_of_Loan'] = pd.to_numeric(df['Num_of_Loan'], errors='coerce').fillna(0).astype(int)

# Num_of_Delayed_Payment (object → int64)
df['Num_of_Delayed_Payment'] = pd.to_numeric(df['Num_of_Delayed_Payment'], errors='coerce').fillna(0).astype(int)

# Changed_Credit_Limit (object → float64)
df['Changed_Credit_Limit'] = pd.to_numeric(df['Changed_Credit_Limit'], errors='coerce')

# Outstanding_Debt (object → float64)
df['Outstanding_Debt'] = df['Outstanding_Debt'].str.replace(r'[^\d.]', '', regex=True).astype(float)

# Amount_invested_monthly (object → float64)
df['Amount_invested_monthly'] = pd.to_numeric(df['Amount_invested_monthly'], errors='coerce')

# Monthly_Balance (object → float64)
df['Monthly_Balance'] = pd.to_numeric(df['Monthly_Balance'], errors='coerce')

# Age (object → int64)
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
age_median = df['Age'].median()  # Compute median of non-NaN values
df['Age'] = df['Age'].fillna(age_median).astype(int)

# Clean Categorical Columns

In [134]:
df['Occupation'] = df['Occupation'].replace("_______", "Unknown")

# Standardize format by converting to title case and removing extra spaces
df['Occupation'] = df['Occupation'].str.title().str.strip()

# Check for duplicates by printing unique values
print("Unique Occupations after cleaning:")
print(df['Occupation'].unique())

# Verify the cleaned column with value counts
print("\nOccupation distribution after cleaning:")
print(df['Occupation'].value_counts())

Unique Occupations after cleaning:
['Scientist' 'Unknown' 'Teacher' 'Engineer' 'Entrepreneur' 'Developer'
 'Lawyer' 'Media_Manager' 'Doctor' 'Journalist' 'Manager' 'Accountant'
 'Musician' 'Mechanic' 'Writer' 'Architect']

Occupation distribution after cleaning:
Occupation
Unknown          7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: count, dtype: int64


In [135]:
df['Credit_Mix'] = df['Credit_Mix'].fillna("Unknown")
# Replace "_" with "Unknown"
df['Credit_Mix'] = df['Credit_Mix'].replace("_", "Unknown")
# Standardize format - convert to title case and strip extra spaces
df['Credit_Mix'] = df['Credit_Mix'].str.title().str.strip()

print("Unique values in Credit_Mix after cleaning:")
print(df['Credit_Mix'].unique())

Unique values in Credit_Mix after cleaning:
['Unknown' 'Good' 'Standard' 'Bad']


In [136]:
df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].fillna("Unknown")
# Replace "NM" with "Unknown"
df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].replace("NM", "Unknown")
# Standardize format - convert to title case and strip extra spaces
df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].str.title().str.strip()

print("Unique values in Payment_of_Min_Amount after cleaning:")
print(df['Payment_of_Min_Amount'].unique())

Unique values in Payment_of_Min_Amount after cleaning:
['No' 'Unknown' 'Yes']


In [137]:
df['Payment_Behaviour']

0         High_spent_Small_value_payments
1          Low_spent_Large_value_payments
2         Low_spent_Medium_value_payments
3          Low_spent_Small_value_payments
4        High_spent_Medium_value_payments
                       ...               
99995     High_spent_Large_value_payments
99996    High_spent_Medium_value_payments
99997     High_spent_Large_value_payments
99998      Low_spent_Large_value_payments
99999                              !@9#%8
Name: Payment_Behaviour, Length: 100000, dtype: object

In [138]:
# Replace NaN values with "Unknown"
df['Payment_Behaviour'] = df['Payment_Behaviour'].fillna("Unknown")
# Replace special characters with "Unknown"
df['Payment_Behaviour'] = df['Payment_Behaviour'].replace("!@9#%8", "Unknown")
# Standardize format - replace underscores with spaces, convert to title case, and strip extra spaces
df['Payment_Behaviour'] = df['Payment_Behaviour'].str.replace("_", " ").str.title().str.strip()

# Check for duplicates by printing unique values
print("Unique values in Payment_Behaviour after cleaning:")
unique_values = df['Payment_Behaviour'].unique()
print(unique_values)

duplicate_mapping = {
    "Low Spent Small": "Low Spent Small Value Payments",
    "High Spent Small": "High Spent Small Value Payments",
    "Low Spent Large": "Low Spent Large Value Payments",
    "Low Spent Medium": "Low Spent Medium Value Payments",
    "High Spent Medium": "High Spent Medium Value Payments",
    "High Spent Large": "High Spent Large Value Payments",
    "Unknown": "Unknown"
}

df['Payment_Behaviour'] = df['Payment_Behaviour'].replace(duplicate_mapping)

# Verify the cleaned column with value counts
print("\nPayment_Behaviour distribution after cleaning:")
print(df['Payment_Behaviour'].value_counts())

Unique values in Payment_Behaviour after cleaning:
['High Spent Small Value Payments' 'Low Spent Large Value Payments'
 'Low Spent Medium Value Payments' 'Low Spent Small Value Payments'
 'High Spent Medium Value Payments' 'Unknown'
 'High Spent Large Value Payments']

Payment_Behaviour distribution after cleaning:
Payment_Behaviour
Low Spent Small Value Payments      25513
High Spent Medium Value Payments    17540
Low Spent Medium Value Payments     13861
High Spent Large Value Payments     13721
High Spent Small Value Payments     11340
Low Spent Large Value Payments      10425
Unknown                              7600
Name: count, dtype: int64


In [139]:
print("Unique values before cleaning:", df['Credit_Score'].unique())

# Standardize the format - strip extra spaces and convert to title case
df['Credit_Score'] = df['Credit_Score'].str.strip().str.title()

# Check unique values after cleaning
print("Unique values after cleaning:", df['Credit_Score'].unique())

Unique values before cleaning: ['Good' 'Standard' 'Poor']
Unique values after cleaning: ['Good' 'Standard' 'Poor']


# Handle Special Columns

In [140]:
# month column 
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

df['Month_number_2'] = pd.Categorical(df['Month'], categories=month_order, ordered=True)
print(df['Month'].head())


0     January
1    February
2       March
3       April
4         May
Name: Month, dtype: object


# Handling Missing Data

# Check for Outliers